# Basic PUMS Analysis with OpenDP

This notebook will be a brief tutorial on doing data analysis within the OpenDP system.

We will start out by setting up our environment -- loading the necessary libraries and establishing the very basic
things we need to know before loading our data (the file path and variable names).

In [ ]:
# load libraries
from opendp.trans import *
from opendp.meas import *
from opendp.core import *
from opendp.typing import *

# establish data information
data_path = os.path.join('.', 'data', 'PUMS_california_demographics_1000', 'data.csv')
var_names = ["age", "sex", "educ", "race", "income", "married", "pid"]

# TODO: Remove column headers
with open(data_path) as input_data:
    data = input_data.read()

### Properties

*TODO* OpenDP architeture description here

Let's examine how we can read and process data within a computation chain. We already have data as a string, read in
from a file.
Here are the first 5 lines of the data:

In [ ]:
print('\n'.join(data.split('\n')[:6]))

Now let's read this data into a computation chain and select the age column. Notice we will need to specify the columns
as integer indices. make_select_column will then give us the nth column as a particular type. In this case, giving
us the first column cast as a string:

In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4, 5]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=0, T=str)
)
res = preprocessor(data)
print(type(res))
print(res)

Age doesn't make sense as a string for our purposes, so let's cast it to an integer:

In [ ]:
# Create a chained computation
preprocessor = (
    # Convert data into Vec<Vec<String>>
    make_split_dataframe(separator=",", col_names=[0, 1, 2]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=0, T=str)
    # Cast the column as Vec<Int>
)
chain = preprocessor >> make_cast(TI=str, TO=int) >> make_impute_constant(0)
print(chain(data)[:10])

Now that we have an age column as integers and we've imputed any missing values, let's clamp the values to a defined
range so that we can quantify our sensitivity for future computations:

In [ ]:
# Create a chained computation
preprocessor = (
    # Convert data into Vec<Vec<String>>
    make_split_dataframe(separator=",", col_names=[0, 1, 2]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=0, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=int) >>
    # Impute missing values to 0
    make_impute_constant(0) >>
    make_clamp(20, 50)
)
res = preprocessor(data)
print(res[:10])

Notice the first element was 0, but the clamp changed it to 20. Similarly, the second element has changed from 59 to 50.

Time to compute our first aggregate statistic. Suppose we want to know the sum of the ages in our dataset.
We can add one more step to our previous computation chain: make_bounded_sum. This will take the result of make_clamp,
and calculate the sum over the domain [20, 50].

In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=0, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=int) >>
    # Impute missing values to 0
    make_impute_constant(0) >>
    # Clamp age values
    make_clamp(20, 50) >>
    make_bounded_sum(lower=20, upper=50)
)

res = preprocessor(data)
print(res)


We may be more interested in the mean age of the data. Then we can take the result of clamp and call
make_bounded_mean over the same domain.

Our bounded mean age is:

In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    # TODO: how to remove column headers?
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=0, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=float) >>
    # Impute missing values to 0
    make_impute_constant(0.) >>
    # Clamp age values
    make_clamp(20., 50.)
    # make_bounded_mean(lower=1000., upper=1_000_000., n=100, T=float)
    # make_base_laplace(scale=1.0)

)

# TOOO: chain these into one process. Currently getting domain mismatch error
# Mike: Once you do the count, you could pull the count before the mean, and then use the count as an input to resize.
res = preprocessor(data)
mean_process = make_bounded_mean(20., 50., n=1000, T=float) >> make_base_laplace(1.)
res = mean_process(res)
print(res)

Privatizing sums
(TODO: Explain geometric mechanism and how it privatizes the sum)

Let's use the geometric mechanism to create a DP release of the sum of all values
in the first column

In [ ]:
def user_function(index, data_type, lower, upper, epsilon):
    preprocessor = (
        # Convert data into Vec<Vec<String>>
        make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
        # Selects a column of df, Vec<str>
        make_select_column(key=0, T=str) >>
        # Cast the column as Vec<Int>
        make_cast(TI=str, TO=int) >>
        # Impute missing values to 0
        make_impute_constant(0) >>
        # Clamp age values
        make_clamp(20, 50) >>
        make_bounded_sum(lower=20, upper=50) >>
        make_base_geometric(scale=1.0)

    )

    res = preprocessor(data)
    print("DP Sum: ", res)

Privatizing counts:

Sometimes you will want to make a DP release of the total number of elements in a data set. For example: How many rows
does our data set have? Below, we will use `make_count` to calculate this number, and `make_base_geometric` to
privatize the value.

In [ ]:
with open(data_path) as input_data:
    data = input_data.read()

In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=4, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=int) >>
    # Impute missing values to 0
    make_impute_constant(0) >>
    # Clamp age values
    make_clamp(1000, 1_000_000) >>
    make_bounded_sum(lower=1000, upper=1_000_000)
    # make_base_laplace(scale=1.0)

)


res = preprocessor(data)
print("Income column sum: ", res)


In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    # TODO: how to remove column headers?
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=4, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=float) >>
    # Impute missing values to 0
    make_impute_constant(0.) >>
    # Clamp age values
    make_clamp(1000., 1_000_000.)
    # make_bounded_mean(lower=1000., upper=1_000_000., n=100, T=float)
    # make_base_laplace(scale=1.0)

)

# TOOO: chain these into one process. Currently getting domain mismatch error
res = preprocessor(data)
mean_process = make_bounded_mean(1000., 1_000_000., n=10, T=float)
res = mean_process(res)
print("Income bounded mean: ", res)

In [ ]:
query = make_bounded_mean(lower=0., upper=10., n=9)
print("Bounded mean: ", query([float(x) for x in range(0,10)]))

In [ ]:
query = make_bounded_mean(lower=0., upper=10., n=9)
print("Bounded mean: ", query([float(x) for x in range(0,10)]))

In [ ]:
with open(data_path) as input_data:
    data = input_data.read()

In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=4, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=int) >>
    # Impute missing values to 0
    make_impute_constant(0) >>
    # Clamp age values
    make_clamp(1000, 1_000_000) >>
    make_bounded_sum(lower=1000, upper=1_000_000)
    # make_base_laplace(scale=1.0)

)


res = preprocessor(data)
print("Income column sum: ", res)


In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    # TODO: how to remove column headers?
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=4, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=float) >>
    # Impute missing values to 0
    make_impute_constant(0.) >>
    # Clamp age values
    make_clamp(1000., 1_000_000.)
    # make_bounded_mean(lower=1000., upper=1_000_000., n=100, T=float)
    # make_base_laplace(scale=1.0)

)

# TOOO: chain these into one process. Currently getting domain mismatch error
res = preprocessor(data)
mean_process = make_bounded_mean(1000., 1_000_000., n=10, T=float)
res = mean_process(res)
print("Income bounded mean: ", res)

In [ ]:
query = make_bounded_mean(lower=0., upper=10., n=9)
print("Bounded mean: ", query([float(x) for x in range(0,10)]))

In [ ]:
query = make_bounded_mean(lower=0., upper=10., n=9)
print("Bounded mean: ", query([float(x) for x in range(0,10)]))

In [ ]:
with open(data_path) as input_data:
    data = input_data.read()

In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=4, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=int) >>
    # Impute missing values to 0
    make_impute_constant(0) >>
    # Clamp age values
    make_clamp(1000, 1_000_000) >>
    make_bounded_sum(lower=1000, upper=1_000_000)
    # make_base_laplace(scale=1.0)

)


res = preprocessor(data)
print("Income column sum: ", res)


In [ ]:
preprocessor = (
    # Convert data into Vec<Vec<String>>
    # TODO: how to remove column headers?
    make_split_dataframe(separator=",", col_names=[0, 1, 2, 3, 4]) >>
    # Selects a column of df, Vec<str>
    make_select_column(key=4, T=str) >>
    # Cast the column as Vec<Int>
    make_cast(TI=str, TO=float) >>
    # Impute missing values to 0
    make_impute_constant(0.) >>
    # Clamp age values
    make_clamp(1000., 1_000_000.)
    # make_bounded_mean(lower=1000., upper=1_000_000., n=100, T=float)
    # make_base_laplace(scale=1.0)

)

# TOOO: chain these into one process. Currently getting domain mismatch error
res = preprocessor(data)
mean_process = make_bounded_mean(1000., 1_000_000., n=10, T=float)
res = mean_process(res)
print("Income bounded mean: ", res)

In [ ]:
query = make_bounded_mean(lower=0., upper=10., n=9)
print("Bounded mean: ", query([float(x) for x in range(0,10)]))

In [ ]:
query = make_bounded_mean(lower=0., upper=10., n=9)
print("Bounded mean: ", query([float(x) for x in range(0,10)]))